# FEMS API Tutorial

This notebook is meant to demonstrate using the FEMS API to retrieve fuel moisture field samples. The code will be deployed through the python module `NAME.py`. The current goal is to read in 100h and 1000h field samples and adapt an RNN trained on 10h sensor data to forecast 100h FEMC via transfer learning. Longer term, this could be used to model live fuel types, duff, and dead moss.

## Setup

In [8]:
import numpy as np
import pandas as pd

# Local modules
import src.fems_api as fems
from src.utils import read_yml

## Metadata

The module has functions to get the following metadata related to the project. The code queries the API to get all available fields. The intended use of the functions is to query and build metadata files and save to a stash. The stash creation could be rerun on different systems or when the API updates.
- All available sampling locations. Stashed in `data/fems_sts.xslx`
    - Sites can be filtered by spatial bounding box, see `etc/gaccs.yaml` for bounding boxes for NIFC GACCs
- All available fuel types. Stashed in `etc/fuels.yaml`

NOTE: the spatial bounding boxes were used from `wrfxpy` `rtma_cycler`. The actual GACCs are not rectangular, so the bounding boxes buffer the actual areas. Thus, using the bbox values from `etc/gaccs.yaml` will collect some sampling locations that are designated as other GACCs than the one requested.

In [2]:
sts = fems.get_all_sites(stash_path = "data/fems_sts.xlsx")
fuels = fems.get_fuel_types(stash_path = "etc/fuels.yaml")

Saving stations to data/fems_sts.xlsx


In [3]:
sts.head()

,siteId,siteName,areaId,areaName,stateId,stateName,groupId,groupName,siteStartDate,active,...,elevation,timeZone,timeZoneOffset,mapWidth,modifiedTime,modifiedBy,createdTime,createdBy,firstSampleDate,latestSampleDate
0,210,ZZZ - CIF Monica,SWCC,Southwest,NM,New Mexico,165,USFS - Cibola NF,2013-05-22T05:00Z,0,...,9635,MST,-7,None,2025-05-16T22:17:21.088053Z,Kyrk Barron,None,None,2013-05-20T05:00:00.000Z,2018-07-06T05:00:00.000Z
1,1369,ZZZ - CIF Doc Long,SWCC,Southwest,NM,New Mexico,165,USFS - Cibola NF,2010-05-12T05:00Z,0,...,7587,MST,-7,None,2025-06-09T21:02:01.401645Z,Dennis Carril,None,None,2011-04-14T05:00:00.000Z,2016-07-24T05:00:00.000Z
2,1368,ZZZ - CAF Portrero,SWCC,Southwest,NM,New Mexico,153,USFS - Carson NF,2014-01-03T06:00Z,1,...,7970,MST,-7,None,2024-03-15T19:50:26.225179Z,EA,None,None,2019-05-28T05:00:00.000Z,2020-04-28T05:00:00.000Z
3,1367,ZZZ - CAF Pine Canyon,SWCC,Southwest,NM,New Mexico,153,USFS - Carson NF,2014-01-03T06:00Z,1,...,7790,MST,-7,None,2024-03-15T19:50:26.225179Z,EA,None,None,2019-05-28T05:00:00.000Z,2020-04-22T05:00:00.000Z
4,1366,ZZZ - CAF MDLV,SWCC,Southwest,NM,New Mexico,153,USFS - Carson NF,2012-07-05T05:00Z,1,...,7700,MST,-7,None,2024-03-15T19:50:26.225179Z,EA,None,None,2012-04-30T05:00:00.000Z,2019-08-19T05:00:00.000Z


In [6]:
print(fuels["100-Hour"])

{'fuel_id': 3, 'fuel_type': '100-Hour', 'category': 'Dead', 'scientific_name': None, 'modified_time': '2023-12-20T20:00Z', 'modified_by': None, 'created_time': None, 'created_by': None}


In [11]:
bbox = read_yml("etc/gaccs.yaml", subkey="Rocky Mountain GACC")["bbox"]
bbox

[37, -111, 46, -95]

In [13]:
sts = fems.get_sites_in_bbox(bbox = bbox, stash_path = "data/fems_sts.xlsx")
sts

Getting all available field names from url: https://fems.fs2c.usda.gov/fuelmodel/apis/graphql
Query: 
    query {
      __type(name: "SiteDTO") {
        fields {
          name
        }
      }
    }
    
Response: <Response [200]>
Number of Fields: 34
Saving stations to data/fems_sts.xlsx
Filtering Stations to Bounding Box: [37, -111, 46, -95]
Stations found within bbox: 334


,siteId,siteName,areaId,areaName,stateId,stateName,groupId,groupName,siteStartDate,active,...,elevation,timeZone,timeZoneOffset,mapWidth,modifiedTime,modifiedBy,createdTime,createdBy,firstSampleDate,latestSampleDate
15,1356,Yellowstone,GBCC,Great Basin,UT,Utah,203,Ashley NF,2007-08-30T05:00Z,1,...,7779,MST,-7,None,2024-03-15T19:50:26.225179Z,EA,None,None,2004-06-13T05:00:00.000Z,2025-08-09T00:00:00.000Z
19,1603,Yellow Bear,RMCC,Rocky Mountain,SD,South Dakota,432,Pine Ridge Agency,2024-06-27T06:00Z,1,...,3209,MST,-7,None,2024-12-06T21:20:47.158966Z,Ronald Bradshaw,2024-12-05T19:19:06.829657Z,Ronald Bradshaw,2024-06-27T18:00:00.000Z,2024-07-17T16:00:00.000Z
20,1353,WYSHF Rock Creek,RMCC,Rocky Mountain,WY,Wyoming,219,Shoshone NF,2023-06-20T05:00Z,1,...,8615,MST,-7,None,2024-03-15T19:50:26.225179Z,EA,None,None,2020-05-21T18:00:00.000Z,2025-08-26T11:00:00.000Z
21,1352,WYSHF Elkhorn,RMCC,Rocky Mountain,WY,Wyoming,219,Shoshone NF,2023-06-20T05:00Z,1,...,8115,MST,-7,None,2024-03-15T19:50:26.225179Z,EA,None,None,2013-07-19T17:00:00.000Z,2025-09-08T12:00:00.000Z
23,1350,WY Hall Creek,RMCC,Rocky Mountain,WY,Wyoming,220,Wind River-Bighorn Basin District,2008-06-26T05:00Z,1,...,6350,MST,-7,None,2024-03-15T19:50:26.225179Z,EA,None,None,2002-05-09T05:00:00.000Z,2025-09-22T00:00:00.000Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1566,30,Antelope Butte,RMCC,Rocky Mountain,WY,Wyoming,221,Bighorn NF,2008-07-15T05:00Z,1,...,8530,MST,-7,None,2024-03-15T19:50:26.225179Z,EA,None,None,2007-07-15T05:00:00.000Z,2025-09-19T00:00:00.000Z
1568,28,Anderson Ridge,RMCC,Rocky Mountain,WY,Wyoming,223,Wyoming High Desert District,2008-07-15T05:00Z,1,...,7776,MST,-7,None,2024-03-15T19:50:26.225179Z,EA,None,None,2008-06-16T05:00:00.000Z,2025-06-24T00:00:00.000Z
1579,16,Adams,RMCC,Rocky Mountain,SD,South Dakota,182,Black Hills NF,2008-06-10T05:00Z,1,...,5820,MST,-7,None,2024-03-15T19:50:26.225179Z,EA,None,None,2005-04-13T05:00:00.000Z,2025-09-25T12:00:00.000Z
1587,1486,7E Ranch,RMCC,Rocky Mountain,WY,Wyoming,223,Wyoming High Desert District,2020-06-01T06:00Z,1,...,7265,MST,-7,None,2024-07-09T19:25:14.312038Z,Chase Lane,2024-07-09T19:25:14.312035Z,Chase Lane,2024-06-27T18:00:00.000Z,2024-08-06T18:00:00.000Z


## Retrieval Example
- Define time period and list of stations (e.g. stations inside a bbox)
- Retrieve sample IDs given those parameters
- Retrieve all available 100h, 1000h, and Duff field samples

In [38]:
stids = sts.siteId.to_list()
print(f"Number of stations queried: {len(stids)}")

fm_params = {
    "startDate": "2025-01-01T00:00:00+00",
    "endDate": "2025-10-28T23:00:00+00",
    "siteIds": [1352, 1348],
    "fuelTypes": ["100-Hour", "1000-Hour", "Duff"],
}

Number of stations queried: 334


In [39]:
results = fems.get_fuel_data(
        fm_params
    )

In [40]:
results

,fuel_sample_id,site_id,fuel,sub_category,sample,subSampleCount,method_type,status,sample_average_value
0,377211,1352,"{'fuel_type': '100-Hour', 'category': 'Dead'}",None,2025-09-08T12:00Z,1,None,Submitted,10.0
1,375506,1352,"{'fuel_type': '100-Hour', 'category': 'Dead'}",None,2025-08-25T12:00Z,1,None,Submitted,11.0
2,372877,1352,"{'fuel_type': '100-Hour', 'category': 'Dead'}",None,2025-08-11T11:00Z,1,None,Submitted,8.0
3,370681,1352,"{'fuel_type': '100-Hour', 'category': 'Dead'}",None,2025-07-28T21:00Z,1,None,Submitted,7.0
4,368236,1352,"{'fuel_type': '100-Hour', 'category': 'Dead'}",None,2025-07-15T11:00Z,1,None,Submitted,8.0
5,370687,1352,"{'fuel_type': '100-Hour', 'category': 'Dead'}",None,2025-06-30T18:00Z,1,None,Submitted,9.0
6,364207,1352,"{'fuel_type': '100-Hour', 'category': 'Dead'}",None,2025-06-17T17:00Z,1,None,Submitted,10.0
7,377212,1352,"{'fuel_type': '1000-Hour', 'category': 'Dead'}",None,2025-09-08T12:00Z,1,None,Submitted,11.0
8,375507,1352,"{'fuel_type': '1000-Hour', 'category': 'Dead'}",None,2025-08-25T12:00Z,1,None,Submitted,12.0
9,372878,1352,"{'fuel_type': '1000-Hour', 'category': 'Dead'}",None,2025-08-11T11:00Z,1,None,Submitted,9.0


## Other

In [ ]:
fm_params = {
    "startDate": "2025-01-01T00:00:00+00",
    "endDate": "2025-08-31T23:00:00+00",
    # "bbox": [37, -111, 46, -95],
    "siteIds": [319],
    "fuelTypes": ["Moss, Dead"],
}

In [ ]:
fems.get_fuel_data_of_sites(
        fm_params
    )